<a href="https://colab.research.google.com/github/Vedalaxman/Audio_to_Text/blob/main/Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-generativeai

In [ ]:
api_key = ''

In [ ]:
import os
import google.generativeai as genai

PROMPT_TEMPLATE = """
You are an expert medical scribe. Your task is to take a raw transcription of a doctor-patient consultation and structure it according to the Indian National EHR Standard format.

Analyze the following transcription and extract the information into the sections below.
- If information for a section is not present in the transcript, write "Not mentioned."
- Do not invent any information that is not explicitly stated in the transcription.

**[Patient Identification]**
- Name:
- Age:
- Gender:

**[Chief Complaints]**
-

**[History of Present Illness]**
-

**[Past Medical/Surgical History]**
-

**[Medication History]**
-

**[Allergies]**
-

**[Family History]**
-

**[Physical Examination Findings]**
-

**[Provisional Diagnosis / Assessment]**
-

**[Care Plan / Treatment]**
-

Here is the raw transcription to format:
---
{raw_transcription_text}
---
"""

def configure_api(api_key):
    genai.configure(api_key=api_key)

def format_transcription(transcription_text: str) -> str:
    model = genai.GenerativeModel('gemini-2.0-flash')
    final_prompt = PROMPT_TEMPLATE.format(raw_transcription_text=transcription_text)
    response = model.generate_content(final_prompt)
    return response.text

if __name__ == "__main__":
    configure_api(api_key )

    audio_file_path = "/content/WhatsApp Audio 2025-06-10 at 18.46.02_dde6187b.mp3"

    print(f"Uploading file: {audio_file_path}")
    audio_file = genai.upload_file(path=audio_file_path)
    print(f"Completed upload: {audio_file.name}")

    model = genai.GenerativeModel(model_name="gemini-2.0-flash")
    prompt = "Transcribe this audio file and provide the full text."

    print("Generating content...")
    response = model.generate_content([prompt, audio_file])

    transcription_text = response.text

    print("\n--- Raw Transcription ---")
    print(transcription_text)

    print("\n--- Formatted EHR Output ---")
    formatted_output = format_transcription(transcription_text)
    print(formatted_output)


In [ ]:
!pip install faster-whisper
#run this always

In [ ]:
from faster_whisper import WhisperModel
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

In [ ]:
model = WhisperModel("large", device=device, compute_type=compute_type)

print(f"Loaded on {device} with compute_type={compute_type}")

In [ ]:
segments, info = model.transcribe("/content/WhatsApp Audio 2025-06-10 at 18.46.02_dde6187b (1).mp3", beam_size=10)

In [ ]:
text = []

In [ ]:
text = []
for segment in segments:
    text.append(f"{segment.text}")

In [ ]:
text

In [ ]:
transcription = ' \n'.join(text)

In [ ]:
print(transcription)

In [ ]:
!pip install -q transformers accelerate

In [ ]:
transcription = '''
 Please have a seat here. What's the problem?
 I have a terrible stomach ache.
 Do you have diarrhea?
 Yes, I do.
 Do you have any other symptoms?
 Yes, I feel sick.
 You mean you feel nauseous?
 That's right. I feel like vomiting.
 And right now I feel dizzy, too.
 When did the symptoms start?
 This morning. Yesterday evening I ate something raw.
 All right. Please take off your clothes to the waist and lie down there.
 Just tell me if it hurts when I do this.
 It doesn't hurt.
 Ouch! It hurts there.
 Okay. Let's hope it's just indigestion,
 but we'll need to run some diagnostic tests to be sure.
 We'll run a blood test and we'll also need a urine sample.
 Can you give me something for the time being?
 Yes, I'll give you a prescription for indigestion tablets.
'''

In [ ]:
PROMPT_TEMPLATE = """
You are an expert medical scribe. Your task is to take a raw transcription of a doctor-patient consultation and structure it according to the Indian National EHR Standard format.

Analyze the following transcription and extract the information into the sections below.
- If information for a section is not present in the transcript, write "Not mentioned."
- Do not invent any information that is not explicitly stated in the transcription.

**[Patient Identification]**
- Name:
- Age:
- Gender:

**[Chief Complaints]**
-

**[History of Present Illness]**
-

**[Past Medical/Surgical History]**
-

**[Medication History]**
-

**[Allergies]**
-

**[Family History]**
-

**[Physical Examination Findings]**
-

**[Provisional Diagnosis / Assessment]**
-

**[Care Plan / Treatment]**
-

Here is the raw transcription to format:
---
{raw_transcription_text}
---
"""

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model_id = "HuggingFaceH4/zephyr-1.3b-alpha"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)


prompt = PROMPT_TEMPLATE.format(raw_transcription_text=transcription)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
response = generator(prompt, max_new_tokens=512, temperature=0.3)[0]["generated_text"]

In [ ]:
prompt = PROMPT_TEMPLATE.format(raw_transcription_text=transcription)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
response = generator(prompt, max_new_tokens=512, temperature=0.3)[0]["generated_text"]

In [ ]:
print(response)